In [1]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import difflib
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
dems = pd.read_csv('../datasets/dem_candidates.csv')
fec = pd.read_csv('../datasets/fec-house.csv')
brookings = pd.read_csv('../datasets/brookings.csv')

In [3]:
def count_candidates_by_district(df, district_column, new_column_name):
    """
    Adds a column to the DataFrame with the count of candidates in each district.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the candidates and districts.
    district_column (str): The name of the column with district information.
    new_column_name (str): The name of the new column to hold the counts.

    Returns:
    pd.DataFrame: The original DataFrame with an additional column for candidate counts per district.
    """
    # Calculate the number of candidates in each district
    district_counts = df.groupby(district_column).size().reset_index(name=new_column_name)
    
    # Merge the counts back into the original DataFrame
    df_merged = df.merge(district_counts, on=district_column)

    return df_merged

In [4]:
dems = count_candidates_by_district(dems, 'District', 'total_runners') #run the above func to get count of ppl in each race and add that as a new col
dems = dems[dems['total_runners'] > 1] # filter out all the rows where only 1 person ran. 
dems = dems[dems['Race Type'] == 'Regular'] # filter out all special races due to repetitions and different circumstances
dems = dems[dems['Office Type'] == 'Representative'] #just house

dems = dems.rename(columns={'Warren Endorsed? ': 'Warren Endorsed?', 'Guns Sense Candidate?': 'Gun Sense Candidate?'})
endorsement_cols = dems.columns[pd.Series(dems.columns).str.contains('Endorsed')].values
endorsement_cols = np.append(endorsement_cols, 'Gun Sense Candidate?')
endorsement_cols = np.append(endorsement_cols, 'Party Support?')
# endorsement_cols = np.append(endorsement_cols, 'No Labels Support?')
dems[endorsement_cols] = dems[endorsement_cols].replace({'No': 0, 'Yes': 1})
dems['Total Endorsements'] = dems[endorsement_cols].sum(axis=1)
dems['Endorsed'] = (dems['Total Endorsements'] > 0).astype(int)

dems['Won Primary'] = (dems['Won Primary'] == 'Yes').astype(int)
dems['Primary Status'] = (dems['Primary Status'] == 'Advanced').astype(int) # same as Won Primary but no missing values

dems['Race'] = (dems['Race'] == 'Nonwhite').astype(int)
dems['Veteran?'] = (dems['Veteran?'] == 'Yes').astype(int)
dems['LGBTQ?'] = (dems['LGBTQ?'] == 'Yes').astype(int)
dems['Self-Funder?'] = (dems['Self-Funder?'] == 'Yes').astype(int)
dems['STEM?'] = (dems['STEM?'] == 'Yes').astype(int)
dems['Elected Official?'] = (dems['Elected Official?'] == 'Yes').astype(int)

dems = dems.reset_index().drop(columns=['index'])

In [5]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=80, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].str.extract('([\w\s]*)')[0].tolist()
    
    m = df_1[key1].apply(lambda x: process.extractOne(x, s, scorer=fuzz.token_sort_ratio, score_cutoff=threshold))    
    df_1['matches'] = m.str[0]
    
    return df_1

In [6]:
dems['first'] = dems['Candidate'].str.lower().str.split(' ', n=1, expand=True)[0]
lastnames = dems['Candidate'].str.lower().str.split(' ', n=1, expand=True)[1].str.findall('^([a-z]*)$|(?<=[\s\t])(\w+\-*)$|^(\w+\-*)(?=[\s\t])').str[-1].astype(str).str.extract('(\w+)')
dems['last'] = lastnames[0]
dems['firstlast'] = dems['first'] + ' ' + dems['last']

fec['first'] = fec['name'].str.lower().str.extract(', (\w+\s*\w+)')[0]
fec['last'] = fec['name'].str.lower().str.extract('(\w+\s*\w+)')[0]
fec['firstlast'] = fec['first'] + ' ' + fec['last']

In [7]:
# manually correcting matching errors
matches = fuzzy_merge(dems, fec, 'Candidate', 'firstlast', threshold=60, limit=1)
matches.loc[19, 'matches'] = 'michael james barkley'
matches.loc[63, 'matches'] = 'jeffrey malloy'
matches.loc[68, 'matches'] = 'mark joaquin williams'
matches.loc[83, 'matches'] = 'michael nesmith'
matches.loc[84, 'matches'] = 'charles joseph enderlin'
matches.loc[85, 'matches'] = 'melvin russell oliver'
matches.loc[97, 'matches'] = 'joshua braxton mccall'
matches.loc[110, 'matches'] = 'michael william smith'
matches.loc[117, 'matches'] = 'david michael gill'
matches.loc[156, 'matches'] = 'joseph william mackey'
matches.loc[169, 'matches'] = 'joshua alexander williamson'
matches.loc[170, 'matches'] = 'kenneth j lave'
matches.loc[198, 'matches'] = 'reginald leonard thomas'
matches.loc[203, 'matches'] = 'michael bernard brown'
matches.loc[225, 'matches'] = 'william r wild'
matches.loc[227, 'matches'] = 'coleman alexander young'
matches.loc[232, 'matches'] = 'weldon frederick wooden'
matches.loc[241, 'matches'] = 'christopher e smith'
matches.loc[246, 'matches'] = 'william c haas'
matches.loc[285, 'matches'] = 'fernando cano'
matches = matches.drop(index=[315]).reset_index().drop(columns=['index'])
matches.loc[320, 'matches'] = 'john dennis love'
matches.loc[432, 'matches'] = 'eric feigl'
matches.loc[475, 'matches'] = 'brandon pendarvis brown'
matches.loc[478, 'matches'] = 'john timothy davis'
matches.loc[481, 'matches'] = 'mark anthony ali'
matches.loc[483, 'matches'] = 'stephen harrison lough'
matches.loc[484, 'matches'] = 'william e hopkins'
matches.loc[505, 'matches'] = 'madeline katherine eden'
matches.loc[511, 'matches'] = 'virginia louise leeder'
matches.loc[535, 'matches'] = 'stephen keith brown ii'
matches.loc[543, 'matches'] = 'joshua andrew imhoff'
matches.loc[544, 'matches'] = 'edward allen'
matches.loc[546, 'matches'] = 'christopher michael perri'
matches.loc[593, 'matches'] = 'elizabeth fletcher'
matches.loc[594, 'matches'] = 'kut frederick weiland'

In [8]:
merged = pd.merge(matches.drop(columns=['first', 'last', 'firstlast']), fec.drop(columns=['first', 'last']), how='left', 
        left_on='matches', right_on='firstlast').drop(columns=['office_full', 'party', 'party_full', 'state', 'district',
       'district_number', 'election_districts', 'election_years', 'cycles', 
       'candidate_status', 'incumbent_challenge', 'incumbent_challenge_full',
       'load_date', 'address_city', 'first_file_date', 'last_file_date', 'last_f2_date', 
       'candidate_id', 'two_year_period', 'candidate_election_year', 'address_state', 
       'address_street_1', 'address_street_2', 'candidate_inactive', 'active_through', 
       'disbursements', 'cash_on_hand_end_period', 'debts_owed_by_committee',
       'individual_itemized_contributions', 'office', 'address_zip', 'name', 'matches', 
       'transfers_from_other_authorized_committee',
       'other_political_committee_contributions', 'firstlast',
       'candidate_id.1', 'election_year', 'cycle', 'is_election', 'coverage_start_date', 'coverage_end_date', 
       'federal_funds_flag', 'has_raised_funds', 'party.1', 'office.1', 'candidate_inactive.1', 'state.1', 
       'district.1', 'district_number.1', 'state_full'])
merged = merged[merged['Office Type'] == 'Representative']

In [9]:
brookings = brookings[brookings['Candidate.Party'] == 'Democrat']
brookings = brookings[brookings['Incumbent'] != 'Candidate is an incumbent Member of Congress'].drop(
            columns=['Unnamed: 0', 'Incumbent', 'Candidate.Party', 'Candidate.Website.URL', 'Freshman.Member', 'Trump.Mention', 'Obama.Mention', 'Sanders.Mention', 
                    'Clinton.Mention', 'Special.Counsel.Mention', 'Travel.Ban.Mention', 'Primary.Outcome', 'Primary.Runoff.Outcome'], inplace=False)

In [10]:
brookings['firstlast'] = (brookings['Candidate.First.Name'].str.lower() + ' ' + brookings['Candidate.Last.Name'].str.lower()).str.replace('.', '', regex=True).str.replace("\'", '', regex=True).str.replace('-', '', regex=True)
merged['lw_Candidate'] = merged['Candidate'].str.lower().str.replace('.', '', regex=True).str.replace("\'", '', regex=True).str.replace('-', '', regex=True)

final_merge = fuzzy_merge(merged, brookings, 'lw_Candidate', 'firstlast', threshold=70, limit=2)
final_merge.loc[170, 'matches'] = 'jasen lave'
final_merge.loc[234, 'matches'] = 'zigmond zozicki'
final_merge.loc[246, 'matches'] = 'william haas'
final_merge.loc[269, 'matches'] = 'd donaldson'
final_merge.loc[350, 'matches'] = 'liubov shirley'
final_merge.loc[424, 'matches'] = 'james crary'
final_merge.loc[484, 'matches'] = 'william hopkins'
final_merge.loc[509, 'matches'] = 'tawana waltercadien'
final_merge.loc[524, 'matches'] = 'j jones'
final_merge.loc[545, 'matches'] = 'edward allen'
final_merge = final_merge.drop(index=529, inplace=False).reset_index().drop(columns=['index'], inplace=False)

In [11]:
all_merged = pd.merge(final_merge, brookings, how='left', left_on='matches', right_on='firstlast').drop(index=629).reset_index().drop(columns=['index'])

In [13]:
all_merged

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Partisan Lean,...,Position.on.Handling.Terrorism.Abroad,Position.on.Russia,Party.Category,Party.Category.1,Incumbency,Female,Democrat,Republican,SinglePayer,firstlast
0,Lizzetta Hill McConnell,AL,U.S. House Alabama District 1,Representative,Regular,6/5/18,0,None,None,-30.680000,...,Candidate provides no information,Candidate provides no information,Establishment Democrat,NaN,0,1.0,1,0,NaN,lizetta mcconnell
1,Robert Kennedy Jr.,AL,U.S. House Alabama District 1,Representative,Regular,6/5/18,1,None,On the Ballot,-30.680000,...,Candidate provides no information,Candidate provides no information,Establishment Democrat,NaN,0,0.0,1,0,NaN,robert kennedy
2,Audri Scott Williams,AL,U.S. House Alabama District 2,Representative,Regular,6/5/18,0,None,None,-33.080002,...,Candidate provides no information,Candidate provides no information,Progressive Democrat,NaN,0,1.0,1,0,"Yes, candidate supports universal healthcare r...",audri williams
3,Tabitha Isner,AL,U.S. House Alabama District 2,Representative,Regular,6/5/18,1,None,On the Ballot,-33.080002,...,Candidate provides no information,Candidate provides no information,Progressive Democrat,NaN,0,1.0,1,0,"Yes, candidate supports universal healthcare r...",tabitha isner
4,Adia McClellan Winfrey,AL,U.S. House Alabama District 3,Representative,Regular,6/5/18,0,None,None,-33.660000,...,Candidate provides no information,Candidate provides no information,Other,NaN,0,1.0,1,0,NaN,adia winfrey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,Talley Sergent,WV,U.S. House West Virginia District 2,Representative,Regular,5/8/18,1,None,On the Ballot,-35.330002,...,Candidate provides no information,Candidate provides no information,Other,NaN,0,1.0,1,0,NaN,talley sergent
628,Janice Hagerman,WV,U.S. House West Virginia District 3,Representative,Regular,5/8/18,0,None,None,-47.480000,...,Candidate provides no information,Candidate provides no information,Establishment Democrat,NaN,0,1.0,1,0,NaN,janice hagerman
629,Paul Davis,WV,U.S. House West Virginia District 3,Representative,Regular,5/8/18,0,None,None,-47.480000,...,Candidate provides no information,Candidate provides no information,Establishment Democrat,NaN,0,0.0,1,0,NaN,paul davis
630,Richard Ojeda,WV,U.S. House West Virginia District 3,Representative,Regular,5/8/18,1,None,On the Ballot,-47.480000,...,Candidate supports calls for increased America...,Candidate provides no information,Establishment Democrat,NaN,0,0.0,1,0,NaN,richard ojeda


In [ ]:
all_merged.to_csv('all_merged.csv', index=False)